<h1><center> ENSF 519.01 Applied Data Scince </center></h1>
<h2><center> Final Project </center></h2>
<h3><center> Oscar Predictions from Movie Scripts </center></h3>
<p><center> Mark Ricalde, Paul Chen, Brian Pho, Harjee Johal <center></p>

# Phase 1

## Data Preprocessing

In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")

def clean_text(text):
    #Lower casing the text
    text = text.lower()
    
    #Removing punctuation
    translation = text.maketrans("", "", string.punctuation)
    text = text.translate(translation)
    re.sub("\s\s+", " ", text)
    
    #Removing stop words
    text = " ".join([word for word in text.split() if word not in ENGLISH_STOP_WORDS])
    
    #Stemming the words
    text = " ".join([stemmer.stem(word) for word in text.split()])

    return text


# Load dataset
data = pd.read_csv('oscar_data.csv') # TODO - Replace dummy csv with real merged csv

# Cleaning the script
data["script"] = data["script"].apply(clean_text)

# Features/Labels for classifying whether a movie will win an oscar
features_1 = data.loc[:, data.columns != 'winner']
labels_1   = data["winner"]

# Features/Labels for classifying what category a movie will be in
features_2 = data.loc[:, data.columns != 'category']
labels_2   = data["category"]
   
# Splitting the data set
xtrain, xtest, ytrain, ytest = train_test_split(features_1, labels_1, stratify=labels_1, test_size=0.2)

# Vectorizing the training set
# TODO - Only vectorize textual features
# tfidf_vect = TfidfVectorizer().fit(xtrain)
# xtrain     = tfidf_vect.transform(xtrain)


KeyError: 'script'

## Visualization and Exploratory Data Analysis

# Phase 2

## Discriminative Models

## Generative Models

## Clustering Models

## Machine Learning Models

### Get imports

In [3]:
# Common imports
import numpy as np
import pandas as pd
import tensorflow as tf

### Get data and labels

In [4]:
from keras.datasets import imdb

(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=max_features)
print(input_train)
print(y_train)

Using TensorFlow backend.


NameError: name 'max_features' is not defined

In [5]:
# Get scripts data
script_csv_path = "./processed_data/script_data.csv"
script_data = pd.read_csv(script_csv_path)
script_data = script_data.drop(columns=["release_year"])
# display(script_data)

# Get Oscar labels
oscar_labels_path = "./processed_data/oscar_data.csv"
oscar_label = pd.read_csv(oscar_labels_path).rename(columns={"entity": "title", "category": "nominated", "winner": "won"})
oscar_label = oscar_label.drop(columns=["year"])
oscar_label["nominated"] = True
# display(oscar_label)

# Merge script and Oscar data
script_oscar_df = pd.merge(script_data, oscar_label, how="outer", on="title")
script_oscar_df = script_oscar_df.dropna(subset=['script'])
script_oscar_df = script_oscar_df.fillna(False)
# display(script_oscar_df)

script_oscar_df['nominated'].astype('bool')
script_oscar_df['won'].astype('bool')
# print(script_oscar_df.dtypes)
# display(script_oscar_df)

script_oscar_df = script_oscar_df[script_oscar_df['nominated'] == script_oscar_df['won']].drop_duplicates()

script_oscar_df = script_oscar_df.drop(columns=['nominated'])
script_oscar_df['won'] = script_oscar_df['won'].astype(int)
script_oscar_df = script_oscar_df.reset_index(drop=True)

display(script_oscar_df)
# print("Won:", np.unique(script_oscar_df['won'], return_counts=True))

,title,script,won
0,When Worlds Collide,"['needl', 'heaven', 'haystack', 'star', 'heave...",1
1,Reds,"['rememb', 'im', 'begin', 'forget', 'peopl', '...",1
2,Undefeated,"['let', 'start', 'right', 'guard', 'shot', 'lo...",1
3,Seven Brides for Seven Brothers,"['deliv', 'perfectionand', 'dont', 'brag', 'd'...",1
4,Henry V,"['oh', 'muse', 'ascend', 'brightest', 'heaven'...",1
...,...,...,...
3195,The Roommate,"['hi', 'hi', 'sara', 'matthew', 'sara', 'matth...",0
3196,Night of the Comet,"['record', 'time', 'swung', 'univers', 'ellipt...",0
3197,Paddington,"['darkest', 'peru', 'vast', 'unexplor', 'wilde...",0
3198,Jumanji: Welcome to the Jungle,"['ocean', 'wave', 'crash', 'seagul', 'squawk',...",0


### Preprocess data for NN

In [6]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

vocab_size = 10000
max_len = 13000
seed = 0

# One-Hot encode the script
script_oscar_df['one_hot_script'] = script_oscar_df['script'].apply(one_hot, n=vocab_size)
# display(script_oscar_df)

# Split the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(script_oscar_df['one_hot_script'], script_oscar_df['won'], random_state=seed)

# Pad scripts
X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, max_len)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [7]:
print(X_train.shape)
print(y_train.shape)
# print(max([len(x) for x in X_train]))
print(type(X_train))
print(type(y_train))

(2400, 13000)
(2400,)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


### Create Simple RNN model

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

model = Sequential()
model.add(Embedding(vocab_size, 32))
# model.add(SimpleRNN(32, return_sequences=True))
# model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 322,113
Trainable params: 322,113
Non-trainable params: 0
_________________________________________________________________


### Train Simple RNN model

In [16]:
simple_rnn_history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2, verbose=1)

Train on 2400 samples
Epoch 1/5
2400/2400 [==============================] - 1096s 457ms/sample - loss: 0.5151 - acc: 0.7896
Epoch 2/5
2400/2400 [==============================] - 1068s 445ms/sample - loss: 0.4282 - acc: 0.8167
Epoch 3/5
 704/2400 [=======>......................] - ETA: 13:11 - loss: 0.4200 - acc: 0.8676

KeyboardInterrupt: 

### Plot results

In [32]:
import matplotlib.pyplot as plt

def plot_results(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')

    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

plot_results(simple_rnn_history)

### Create LSTM model

In [31]:
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 33        
Total params: 328,353
Trainable params: 328,353
Non-trainable params: 0
_________________________________________________________________


### Train LSTM model

In [39]:
lstm_history = model.fit(input_train, y_train,
    epochs=10,
    batch_size=128,
    validation_split=0.2)

### Plot results

In [ ]:
plot_results(lstm_history)